## Importação das bibliotecas

In [312]:
import numpy as np
import pandas as pd
import plotly
import plotly.offline as py
import yfinance as yf
import datetime
import plotly.graph_objs as go

## Sem esse comando os graficos não sao exibidos no notebook

In [313]:
from plotly.offline import plot, iplot
plotly.offline.init_notebook_mode(connected=True)

## Recebe os inputs do usuário e busca as cotações na base de dados do Yahoo

In [314]:
# Recebe o input do usuario com o nome da ação
stock = input('Digite o código da ação que deseja analisar')
stock = stock + '.SA'
# Recebe do usuário o valor das médias 
num_mm_short = int(input('Digite um valor númerico para a média móvel curta'))
num_mm_long = int(input('Digite um valor numerico para a média móvel longa'))
mean_short = 'MM' + str(num_mm_short)
mean_long = 'MM' + str(num_mm_long)
# Define o lote padrão de 100 ações
capital = int(input('Digite o capital disponível para essa estratégia'))

# Define o periodo de busca do historico de preços
ini_date = datetime.date(2020,1,1)
end_date = datetime.date(2020,12,31)

df_stock_prices = yf.download(stock, start=ini_date, end=end_date, interval='1h')
df_stock_prices.reset_index(drop=False, inplace=True)

df_stock_prices.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-01-02,2.61,2.69,2.61,2.64,2.64,150900
1,2020-01-02,2.64,2.76,2.60,2.74,2.74,349400
2,2020-01-02,2.73,2.84,2.71,2.78,2.78,436500
3,2020-01-02,2.78,2.81,2.76,2.80,2.80,107900
4,2020-01-02,2.80,2.81,2.74,2.76,2.76,138600


## Cria as colunas de média móvel no Data Frame e preenche com zero os campos com 'NaN'

In [315]:
df_stock_prices['MM_SHORT'] = df_stock_prices.Close.rolling(window=num_mm_short).mean()
df_stock_prices['MM_LONG'] = df_stock_prices.Close.rolling(window=num_mm_long).mean()
df_stock_prices['SPREAD_MM'] = df_stock_prices['MM_SHORT'] - df_stock_prices['MM_LONG']

df_stock_prices['Start'] = 0.00
df_stock_prices['Stop'] = 0.00
df_stock_prices['Profit'] = 0.00

df_stock_prices['MM_SHORT'] = df_stock_prices['MM_SHORT'].fillna(0)
df_stock_prices['MM_LONG'] = df_stock_prices['MM_LONG'].fillna(0)
# Excluindo do Data frame os registros com a média móvel zerada pois não dá pra fazer backtest nesses registros
df_stock_prices = df_stock_prices.loc[df_stock_prices['MM_LONG']!=0]
df_stock_prices.head(25)

,Date,Open,High,Low,Close,Adj Close,Volume,MM_SHORT,MM_LONG,SPREAD_MM,Start,Stop,Profit
20,2020-01-07,3.67,3.75,3.61,3.68,3.68,1609500,3.365556,3.067143,0.298413,0.0,0.0,0.0
21,2020-01-07,3.69,3.71,3.53,3.61,3.61,1443900,3.433333,3.113333,0.320000,0.0,0.0,0.0
22,2020-01-07,3.60,3.65,3.57,3.61,3.61,308400,3.488889,3.154762,0.334127,0.0,0.0,0.0
23,2020-01-07,3.60,3.63,3.47,3.47,3.47,663800,3.513333,3.187619,0.325714,0.0,0.0,0.0
24,2020-01-07,3.47,3.55,3.36,3.53,3.53,868700,3.532222,3.222381,0.309841,0.0,0.0,0.0
25,2020-01-08,3.58,3.68,3.50,3.52,3.52,1294500,3.543333,3.258571,0.284762,0.0,0.0,0.0
26,2020-01-08,3.52,3.54,3.40,3.49,3.49,1103000,3.546667,3.291905,0.254762,0.0,0.0,0.0
27,2020-01-08,3.48,3.54,3.45,3.48,3.48,190200,3.560000,3.320476,0.239524,0.0,0.0,0.0
28,2020-01-08,3.48,3.52,3.44,3.46,3.46,190500,3.538889,3.350000,0.188889,0.0,0.0,0.0
29,2020-01-08,3.46,3.51,3.44,3.45,3.45,143800,3.513333,3.378095,0.135238,0.0,0.0,0.0


## Plota o gráfico com o preço de fechamento e as médias móveis

In [316]:
close_prices = go.Scatter(
                            x = df_stock_prices['Date'],
                            y = df_stock_prices['Close'],
                            name = stock + 'Preço de Fechamento',
                            line = dict(color = '#330000'),
                            opacity = 1.0)

close_prices2 = go.Candlestick(
                                x = df_stock_prices['Date'],
                                open = df_stock_prices['Open'],
                                high = df_stock_prices['High'],
                                low = df_stock_prices['Low'],
                                close = df_stock_prices['Close'])

mm_s = go.Scatter(
                    x = df_stock_prices['Date'],
                    y = df_stock_prices['MM_SHORT'],
                    name = 'Média Móvel Curta',
                    line = dict(color = '#B2FF66'),
                    opacity = 1.0)

mm_l = go.Scatter(
                    x = df_stock_prices['Date'],
                    y = df_stock_prices['MM_LONG'],
                    name = 'Média Móvel Longa',
                    line = dict(color = '#FF00FF'),
                    opacity = 1.0)

layout = go.Layout(
                    autosize=False,
                    legend_orientation='h',
                    width=1000,
                    height=400)

data_prices = [close_prices,mm_s,mm_l]

fig = go.Figure(data=data_prices, layout=layout)

py.iplot(fig)

## Faz o backtest da estratégia 

In [317]:
start_price = 0
last_spread = 0

for row in df_stock_prices.itertuples():
    lote = capital / row.Close
    #print(lote)
    if (row.MM_SHORT > row.MM_LONG) & (start_price < 0.01):
        #print(start_price)
        df_stock_prices.at[row.Index, 'Start'] = row.Close
        start_price = row.Close
        print(row.Index)
        print(row.Close)
        print(start_price)
    elif (row.MM_SHORT < row.MM_LONG) & (start_price < 0.01):
        #print(start_price)
        df_stock_prices.at[row.Index, 'Start'] = row.Close
        start_price = row.Close
    elif (row.MM_SHORT > row.MM_LONG) & (start_price > 0):
        #print(start_price)
        if last_spread < 0:
            df_stock_prices.at[row.Index, 'Profit'] = (start_price - row.Close) * lote
            df_stock_prices.at[row.Index, 'Stop'] = row.Close
            df_stock_prices.at[row.Index, 'Start'] = row.Close
            start_price = row.Start
        else:
            df_stock_prices.at[row.Index, 'Start'] = start_price
    elif (row.MM_SHORT < row.MM_LONG) & (start_price > 0):
        #print(start_price)
        if last_spread > 0:
            df_stock_prices.at[row.Index, 'Profit'] = (row.Close - start_price) * lote
            df_stock_prices.at[row.Index, 'Stop'] = row.Close
            df_stock_prices.at[row.Index, 'Start'] = row.Close
            start_price = row.Start
        else:
            df_stock_prices.at[row.Index, 'Start'] = start_price
    last_spread = row.SPREAD_MM

df_stock_prices.head(10)

20
3.680000066757202
3.680000066757202
52
3.2899999618530273
3.2899999618530273
102
3.0199999809265137
3.0199999809265137
133
3.0
3.0
225
2.700000047683716
2.700000047683716
251
2.440000057220459
2.440000057220459
368
0.8999999761581421
0.8999999761581421
429
1.1100000143051147
1.1100000143051147
456
1.600000023841858
1.600000023841858
496
1.559999942779541
1.559999942779541
521
1.559999942779541
1.559999942779541
555
1.4600000381469727
1.4600000381469727
610
1.3300000429153442
1.3300000429153442
653
1.350000023841858
1.350000023841858
668
1.3600000143051147
1.3600000143051147
685
1.3600000143051147
1.3600000143051147
754
1.7400000095367432
1.7400000095367432
827
1.690000057220459
1.690000057220459
836
1.7200000286102295
1.7200000286102295
853
1.7400000095367432
1.7400000095367432


,Date,Open,High,Low,Close,Adj Close,Volume,MM_SHORT,MM_LONG,SPREAD_MM,Start,Stop,Profit
20,2020-01-07,3.67,3.75,3.61,3.68,3.68,1609500,3.365556,3.067143,0.298413,3.68,0.0,0.0
21,2020-01-07,3.69,3.71,3.53,3.61,3.61,1443900,3.433333,3.113333,0.320000,3.68,0.0,0.0
22,2020-01-07,3.60,3.65,3.57,3.61,3.61,308400,3.488889,3.154762,0.334127,3.68,0.0,0.0
23,2020-01-07,3.60,3.63,3.47,3.47,3.47,663800,3.513333,3.187619,0.325714,3.68,0.0,0.0
24,2020-01-07,3.47,3.55,3.36,3.53,3.53,868700,3.532222,3.222381,0.309841,3.68,0.0,0.0
25,2020-01-08,3.58,3.68,3.50,3.52,3.52,1294500,3.543333,3.258571,0.284762,3.68,0.0,0.0
26,2020-01-08,3.52,3.54,3.40,3.49,3.49,1103000,3.546667,3.291905,0.254762,3.68,0.0,0.0
27,2020-01-08,3.48,3.54,3.45,3.48,3.48,190200,3.560000,3.320476,0.239524,3.68,0.0,0.0
28,2020-01-08,3.48,3.52,3.44,3.46,3.46,190500,3.538889,3.350000,0.188889,3.68,0.0,0.0
29,2020-01-08,3.46,3.51,3.44,3.45,3.45,143800,3.513333,3.378095,0.135238,3.68,0.0,0.0


In [318]:
print('O resultado da estratégia é R$ %7.2f' % df_stock_prices['Profit'].sum())

O resultado da estratégia é R$ 7613.49


In [319]:
df_stock_prices.to_excel('cruzamento_mm.xlsx', sheet_name='Cross_MM', index=False)